In [3]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from skimage.measure import find_contours, label, regionprops
from skimage.morphology import binary_dilation, disk
from scipy.spatial.distance import cdist
from scipy.ndimage import gaussian_gradient_magnitude


In [2]:
# Filter out mask noise (little patches outside the main mask)
def filter_out_mask_noise(plant_mask):
    # Identify all independent patches
    labels = label(plant_mask, connectivity=2)
    props = regionprops(labels)
    
    # Select only maximal patch
    idx_max_area = np.argmax([x['area'] for x in props])
    return labels == props[idx_max_area]['label']



# Extract the outside contour of the given mask
def extract_plant_contour(leaf_mask):
    return find_contours(leaf_mask)[0]



# Compute mass center of the given mask
def extract_plant_center(plant_mask):
    v_sum_nonzero = np.nonzero(np.sum(plant_mask, axis=0).flatten())[0]
    h_sum_nonzero = np.nonzero(np.sum(plant_mask, axis=1).flatten())[0]
    return np.array(((v_sum_nonzero[0] + v_sum_nonzero[-1]) / 2.0,
                     (h_sum_nonzero[0] + h_sum_nonzero[-1]) / 2.0), dtype=float).reshape((1, 2))


In [3]:
# Calculate a moving average along a given data vector "a"
# If radius is 2 the moving window has 1+2*radius size (1 for the middle element) 
def moving_average(a, radius, wrap=True):
    # w_sz is the window size
    w_sz = 1 + 2 * radius

    if not wrap:
        # When wrap is false, the window at either end of the data vector
        # does not use values from the other end of the vector
        avg_res = np.zeros_like(a, dtype=float)
        
        # Compute cumulative sum
        a_cumsum = np.zeros((1 + a.size), dtype=float)  # One artificial zero is added at the beginning
        a_cumsum[1:] = np.cumsum(a, dtype=float)

        # Most average values can be computed as differences between the cumulative sum values at locations
        # found at w_sz distance from each other
        avg_res[radius:(a.size - radius)] = (a_cumsum[w_sz:] - a_cumsum[:(a.size - w_sz + 1)]) / w_sz
        
        # Some values at the beginning and at the end need special attention
        avg_res[0] = a[0]
        avg_res[-1] = a[-1]
        for idx in range(1, radius):
            curr_w_sz = 1 + 2 * idx
            avg_res[idx] = a_cumsum[curr_w_sz] / curr_w_sz
            avg_res[a.size - idx - 1] = (a_cumsum[-1] - a_cumsum[a.size - curr_w_sz]) / curr_w_sz
        return avg_res
    
    # For a circular filtering we append the beginning sequence at the end
    # The filtering is easier to calculate based on the cumulative sum
    new_a = np.zeros(a.size + 2 * radius, dtype=a.dtype)
    new_a[radius:-radius] = a
    new_a[:radius] = a[-radius:]
    new_a[-radius:] = a[:radius]
    a_cumsum = np.cumsum(new_a, dtype=float)
    
    return np.roll((a_cumsum[w_sz:] - a_cumsum[:-w_sz])[(2 * radius):], 1) / w_sz



# Find local maxima in a data vector
def find_local_maxima_wrapped(a):
    # Rotate data vector to the left and to the right with one position
    a_shift_l = np.roll(a, -1)
    a_shift_r = np.roll(a, 1)
    
    # Find local maxima
    local_max = np.logical_and(a > a_shift_l, a > a_shift_r)
    
    # Find places where we are sure there are no local maxima
    no_max = np.logical_or(a < a_shift_l, a < a_shift_r)
    
    # Expand the no_max areas when there are plateaus next to them
    for seed_idx in np.nonzero(no_max)[0]:
        
        # Examine plateau to the right (if any)
        curr_plateau_idx = []
        for curr_idx in range(seed_idx + 1, seed_idx + a.size - 1):
            real_idx = int(np.mod(curr_idx, a.size))
            if a[seed_idx] == a[real_idx]:
                curr_plateau_idx.append(real_idx)
                continue

            # Plateau ended and is clearly not a max itself as there is at least one greater value next to it
            no_max[curr_plateau_idx] = True
            break

        # Examine plateau to the leftt (if any)
        curr_plateau_idx = []
        for curr_idx in range(seed_idx - 1, seed_idx - 1 - a.size):
            real_idx = int(np.mod(curr_idx, a.size))
            if a[seed_idx] == a[real_idx]:
                curr_plateau_idx.append(real_idx)
                continue

            # Plateau ended and is clearly not a max itself as there is at least one greater value next to it
            no_max[curr_plateau_idx] = True
            break

    # Each remaining plateau must be a flattened peak.
    # We pick the middle of the plateau as representing the peak.
    max_plateaus = np.nonzero(np.logical_not(np.logical_and(local_max, no_max)))[0]
    curr_plateau_idx = [max_plateaus[0]]
    for sel_idx in range(1, max_plateaus.size):
        if max_plateaus[sel_idx] == 1 + curr_plateau_idx[-1]:
            curr_plateau_idx.append(max_plateaus[sel_idx])
            continue
        local_max[int((curr_plateau_idx[0] + curr_plateau_idx[-1]) / 2.0)] = True
        curr_plateau_idx = [max_plateaus[sel_idx]]
    local_max[int((curr_plateau_idx[0] + curr_plateau_idx[-1]) / 2.0)] = True
    
    return np.nonzero(local_max)[0]



# Find the tip of each leaf and the points on the plant contour where two leaves meet each other
def find_leaf_tips_and_separators(plant_center, plant_contour):
    # Compute the distances from the plant center to the plant contour
    all_contour_coords = np.transpose(np.nonzero(plant_contour))
    contour_dists = cdist(plant_center, all_contour_coords)
    contour_dists_smooth = moving_average(contour_dists, 1)

    # Find leaf tips as local maxima of the computed distances
    init_tips_idx = find_local_maxima_wrapped(contour_dists_smooth)
    
    # Find leaf separators as local minima of the computed distances
    init_separators_idx = find_local_maxima_wrapped(-contour_dists_smooth)
    
    # Create "runs" of consecutive tips or consecutive separators
    extrema_list = dict([(x, True) for x in init_tips_idx] + [(x, false) for x in init_separators_idx])
    key_list = sorted(list(extrema_list.keys()))
    run_list = []
    curr_run = key_list[0]
    for key in key_list[1:]:
        if extrema_list[key] == extrema_list[curr_run[0]]:
            curr_run.append[key]
            continue
        run_list.append(curr_run)
        curr_run = [key]
    if extrema_list[curr_run[0]] == extrema_list[run_list[0][0]]:
        # Last run must be prepended to the first run
        for key in curr_run[::-1]:
            run_list[0].insert(0, key)
    else:
        run_list.append(curr_run)
    # Left column contains tip indexes, right column contains the separator indexes following to the right
    tips_and_separators_idx = np.array((int(len(run_list)), 2), dtype=int)
    
    # Select just one tip between two separators and one separator between two tips
    pair_idx = 0
    for curr_run in run_list:
        if  extrema_list[curr_run[0]]:
            # For each tip sequence select only the one with the maximal distance to the plant center 
            idx_max = np.argmax(contour_dists_smooth[curr_run])
            tips_and_separators_idx[pair_idx, 0] = curr_run[idx_max]
        else:
            # For each separator sequence select only the one with the minimal distance to the plant center 
            idx_min = np.argmin(contour_dists_smooth[curr_run])
            tips_and_separators_idx[pair_idx, 1] = curr_run[idx_min]
            pair_idx = np.mod(1 + pair_idx, tips_and_separators_idx.shape[0])
        
    return tips_and_separators_idx


In [ ]:
# Compute for each pixel which of its neighbours offers the path to the plant center
# along a maximal gradient magnitude
def compute_all_paths_from_plant_center_to_contour(rgb_image, plant_mask, plant_center, plant_contour):
    # Compute gray version of the image
    gray_image = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2GRAY)
    
    # Initialize structures holding the coordinates of the next points
    next_pix_y_towards_center = -1 + np.zeros_like(gray_image, dtype=int)
    next_pix_x_towards_center = -1 + np.zeros_like(gray_image, dtype=int)

    # Initialize structures holding the cumulative gradient magnitude towards the plant center
    cumulative_grad_mag_to_center = np.zeros(gray_image.shape, dtype=float)
    cumulative_grad_mag_to_center[:, :] = -np.inf
    cumulative_grad_mag_to_center[plant_center] = 0
    
    # Compute gradient magnitude
    gradient_magnitude = gaussian_gradient_magnitude(gray_image, sigma=3)
    
    # completed_mask will hold the paths that were already calculated up to the current moment
    completed_mask = np.zeros_like(gray_image, dtype=bool)
    completed_mask[plant_center] = True
    completed_mask_dilated = np.logical_and(binary_dilation(completed_mask, disk(1, dtype=bool)), plant_mask)
    
    # next_ring contains the next layer of pixels for which we calculate the best path
    next_ring = np.logical_xor(completed_mask_dilated, completed_mask)
    while np.any(next_ring):
        ring_coords = np.transpose(np.nonzero(next_ring))

        # For each pixel from the ring we calculate the best path
        for ring_pix_idx in range(ring_coords.shape[0]):
            curr_coords = ring_coords[ring_pix_idx, :]
            min_y = -1
            min_x = -1
            max_neighbour_val = -np.inf
            
            # Inspect each neighbour in order to find the best path
            for y in range(max(0, curr_coords[0] - 1), min(gray_image.shape[0], curr_coords[0] + 2)):
                for x in range(max(0, curr_coords[1] - 1), min(gray_image.shape[1], curr_coords[1] + 2)):
                    # The central pixel is the current pixel, we skip it
                    if y == curr_coords[0] and x == curr_coords[1]:
                        continue
                    
                    # Found a better path to the plant center, save it
                    if cumulative_grad_mag_to_center[y, x] > max_neighbour_val:
                        max_neighbour_val = cumulative_grad_mag_to_center[y, x]
                        min_y = y
                        min_x = x
            # Mark the cost of the best path found
            cumulative_grad_mag_to_center[curr_coords] = max_neighbour_val + gradient_magnitude[curr_coords]

            # Mark the next pixel (towards the plant center) of the best path found
            next_pix_y_towards_center = min_y
            next_pix_x_towards_center = min_x
            
            # Add current pixel to the completed mask
            completed_mask[curr_coords] = True
        
        # Compute next ring of pixels
        completed_mask_dilated = np.logical_and(binary_dilation(completed_mask, disk(1, dtype=bool)), plant_mask)
        next_ring = np.logical_xor(completed_mask_dilated, completed_mask)
        
    return cumulative_grad_mag_to_center, next_pix_y_towards_center, next_pix_x_towards_center



# Compute a list of leaf masks
def extract_leaf_masks(rgb_image, plant_mask, plant_center, plant_contour, tips_and_separators_idx):
    # Compute the best path towards the plant center for each pixel in the image
    (cumulative_grad_mag_to_center, 
     next_pix_y_towards_center, 
     next_pix_x_towards_center) = compute_all_paths_from_plant_center_to_contour(
        rgb_image, plant_mask, plant_center, plant_contour)
    
    # Extract leaf mask for each leaf
    n_leafs = tips_and_separators_idx.shape[0]
    leaf_masks = []
    for i_leaf in range(n_leafs):
        # Find path towards plant center starting from the leaf separators on both sides of the leaf
        _, next_separator_idx = tips_and_separators_idx[i_leaf, :]
        _, prev_separator_idx = tips_and_separators_idx[np.mod(i_leaf-1, n_leafs), :]
        
        # Initialize leaf contour with the plant contour between previous and next separators
        curr_leaf_contour = []
        for plant_contour_idx in range(prev_separator_idx, 1 + next_separator_idx):
            curr_leaf_contour.append(plant_contour[plant_contour_idx, :].tolist())
            
        # Get the path between next separator and plant center and add it to the leaf contour
        curr_pix = plant_contour[next_separator_idx, :]
        while curr_pix != plant_center:
            next_y = next_pix_y_towards_center[curr_pix]
            next_x = next_pix_x_towards_center[curr_pix]
            curr_pix[0, 0] = next_y
            curr_pix[0, 1] = next_x
            curr_leaf_contour.append([next_y, next_x])
            
        # Get the path between previous separator and plant center and add it to the leaf contour
        curr_pix = plant_contour[prev_separator_idx, :]
        while curr_pix != plant_center:
            prev_y = next_pix_y_towards_center[curr_pix]
            prev_x = next_pix_x_towards_center[curr_pix]
            curr_pix[0, 0] = prev_y
            curr_pix[0, 1] = prev_x
            curr_leaf_contour.insert(0, [prev_y, prev_x])
            
        # Add the plant center itself to the leaf contour.
        curr_leaf_contour.insert(0, plant_center)        
        curr_leaf_mask = np.zeros_like(plant_mask)
        
        # Compute leaf mask based on leaf contour
        cv2.fillPoly(curr_leaf_mask, curr_leaf_contour, True)
        leaf_masks.append(curr_leaf_mask)
        
    return leaf_masks



# Main function for extracting leaves from the plant mask
def extract_all_leaves(plant_mask):
    # Eliminate noise from plant mask
    new_plant_mask = filter_out_mask_noise(plant_mask)

    # Extract plant contour
    plant_contour = extract_plant_contour(new_plant_mask)

    # Extract plant center
    plant_center = extract_plant_center(new_plant_mask)
    
    # Extract leaf tips and separators
    tips_and_separators_idx = find_leaf_tips_and_separators(plant_center, plant_contour)
    
    # Extract and return individual leaf masks
    return extract_leaf_masks(rgb_image, new_plant_mask, plant_center, plant_contour, tips_and_separators_idx)



In [ ]:
# This is how to calculate the leaf masks and the number of leaves
leaf_masks = extract_all_leaves(plant_mask)
n_leaves = len(leaf_masks)